In [1]:
#%%
import pandas as pd
import warnings
import os
import datetime
import gc
from tqdm.auto import tqdm
import shutil, random
import pickle
import itertools
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

warnings.simplefilter(action='ignore', category=FutureWarning)
os.umask(0)

# from pandarallel import pandarallel
# pandarallel.initialize(nb_workers=4, progress_bar=True)


18

In [2]:
#%%
def epinum_assign(g):
    g = g.sort_values('date').reset_index(drop=True)
    time_gap = [pd.to_datetime(g['date'].tolist()[x + 1]) - pd.to_datetime(g['date'].tolist()[x]) for x in
                range(len(g['date'].tolist()) - 1)]  # shift로 바꿔보면 좋을듯
    sep = [k for k, v in enumerate(time_gap) if v > pd.Timedelta('72h')]
    sep = sorted(set(sep + [0, len(g.index)]))

    end = 0
    for epi_num, pos in enumerate(sep):
        if pos < len(g.index):
            # print(g.index[end:sep[epi_num+1]+1])
            g.at[g.index[end:sep[epi_num + 1] + 1], 'epi_num'] = epi_num + 1
            end = sep[epi_num + 1] + 1

    g['data_1'] = g['data_1'].astype(str)
    return g

In [3]:
#chunk_iter = iter_chunk_by_id("/opt/project/md_data__with_baby_having_dx_sorted.csv")
base_dir = "./data"
file_name = "md_data__with_baby_having_dx_sorted.csv"
filepath = os.path.join(base_dir, file_name)
df =  pd.read_csv(filepath, \
                  usecols=['baby_id', 'date', 'type', 'data_1', 'data_2', 
                           'gender', 'birthday', 'weight', 'convulsion' ])

#weather_ref = pd.read_csv('./OBS_ASOS_DD_20211209143339.csv')


/Users/mckim/anaconda3/envs/flu_pred/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def convert(seconds):
    #seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return "%02d:%02d:%02d" % (hour, minutes, seconds)
#%%
root_path = f"./v2_episode/{str(datetime.datetime.now()).split(' ')[0]}"  # date만 추출. YYYY-mm-dd 형식

if not (os.path.exists(f'{root_path}/flu')):
    os.makedirs(f'{root_path}/flu')

if not (os.path.exists(f'{root_path}/notflu')):
    os.makedirs(f'{root_path}/notflu')

In [7]:
event_uid = 20000
skip_count = 0
save_count = 0

def post_split_csv_by_event(baby_id, group):
    global event_uid
    global skip_count
    global save_count
    
    # pd.DataFram.groupby 요소에 epi_num 번호를 추가
    g_new = epinum_assign(group)
    
    # group 하나씩 보면서 episode file 생성
    for epi_num, g_epi in g_new.groupby('epi_num'):
        if g_epi.loc[(g_epi['type']==4) & (g_epi['data_1']=='3')].empty:
            skip_count +=1
            continue

        filename = "{}__{}__{}.csv".format(event_uid, baby_id, int(epi_num))
        fullpath = os.path.join(root_path, 'postprocessed', filename)
        
        if event_uid % 1000 == 0:
            print("Saving...:{}".format(fullpath))
            print("save:{} skip:{}".format(save_count, skip_count))
        g_epi.to_csv(fullpath)
        event_uid += 1
        save_count += 1
        
for baby_id, baby_group in tqdm(df.groupby('baby_id')):
    post_split_csv_by_event(baby_id, baby_group)
    

Saving...:./v2_episode/2021-12-16/postprocessed/20000__10044__7.csv
save:0 skip:6
Saving...:./v2_episode/2021-12-16/postprocessed/21000__11614__8.csv
save:1000 skip:4315
Saving...:./v2_episode/2021-12-16/postprocessed/22000__12492__13.csv
save:2000 skip:8741
Saving...:./v2_episode/2021-12-16/postprocessed/23000__13393__1.csv
save:3000 skip:13324
Saving...:./v2_episode/2021-12-16/postprocessed/24000__14375__15.csv
save:4000 skip:18491
Saving...:./v2_episode/2021-12-16/postprocessed/25000__15274__10.csv
save:5000 skip:22649
Saving...:./v2_episode/2021-12-16/postprocessed/26000__16074__2.csv
save:6000 skip:27082
Saving...:./v2_episode/2021-12-16/postprocessed/27000__16839__20.csv
save:7000 skip:31642
Saving...:./v2_episode/2021-12-16/postprocessed/28000__18085__13.csv
save:8000 skip:36775
Saving...:./v2_episode/2021-12-16/postprocessed/29000__19343__1.csv
save:9000 skip:41960
Saving...:./v2_episode/2021-12-16/postprocessed/30000__20513__8.csv
save:10000 skip:46995
Saving...:./v2_episode/2

Saving...:./v2_episode/2021-12-16/postprocessed/110000__196144__9.csv
save:90000 skip:407772
Saving...:./v2_episode/2021-12-16/postprocessed/111000__199086__2.csv
save:91000 skip:411566
Saving...:./v2_episode/2021-12-16/postprocessed/112000__202161__1.csv
save:92000 skip:415253
Saving...:./v2_episode/2021-12-16/postprocessed/113000__205342__9.csv
save:93000 skip:419182
Saving...:./v2_episode/2021-12-16/postprocessed/114000__208512__3.csv
save:94000 skip:423099
Saving...:./v2_episode/2021-12-16/postprocessed/115000__211570__12.csv
save:95000 skip:427100
Saving...:./v2_episode/2021-12-16/postprocessed/116000__214929__7.csv
save:96000 skip:431092
Saving...:./v2_episode/2021-12-16/postprocessed/117000__218315__7.csv
save:97000 skip:435058
Saving...:./v2_episode/2021-12-16/postprocessed/118000__221119__11.csv
save:98000 skip:438532
Saving...:./v2_episode/2021-12-16/postprocessed/119000__222305__1.csv
save:99000 skip:443481
Saving...:./v2_episode/2021-12-16/postprocessed/120000__224499__1.cs

Saving...:./v2_episode/2021-12-16/postprocessed/198000__544604__1.csv
save:178000 skip:704363
Saving...:./v2_episode/2021-12-16/postprocessed/199000__550866__4.csv
save:179000 skip:707026
Saving...:./v2_episode/2021-12-16/postprocessed/200000__556778__2.csv
save:180000 skip:709478
Saving...:./v2_episode/2021-12-16/postprocessed/201000__562824__1.csv
save:181000 skip:711994
Saving...:./v2_episode/2021-12-16/postprocessed/202000__568978__3.csv
save:182000 skip:714593
Saving...:./v2_episode/2021-12-16/postprocessed/203000__575211__1.csv
save:183000 skip:716979
Saving...:./v2_episode/2021-12-16/postprocessed/204000__581431__2.csv
save:184000 skip:719409
Saving...:./v2_episode/2021-12-16/postprocessed/205000__588094__2.csv
save:185000 skip:721751
Saving...:./v2_episode/2021-12-16/postprocessed/206000__594657__4.csv
save:186000 skip:724242
Saving...:./v2_episode/2021-12-16/postprocessed/207000__601418__2.csv
save:187000 skip:726703
Saving...:./v2_episode/2021-12-16/postprocessed/208000__6080

In [8]:
print("save:{} skip:{}".format(save_count, skip_count))

save:226655 skip:788641
